# Module 2 Final Project

* Student name: JAMES SAVORY
* Student pace: PART TIME
* Scheduled project review date/time: JUNE 10, 2019 
* Instructor name: BRANDON LEWIS
* Blog post URL:https://jsavory-94.github.io/module_2_project_rough

# A. Appendix

## I. Summary

The dataset used in this project is the Northwind Traders Database created by Microsoft. It contains data on various aspects of the company such as products, orders, sales, employees, and customers.

This goal of this project is to generate insights from the data that would contribute business value to the company. This will be done by implementing a series of statistical tests to tackle each of our questions identified below. 

1. [Question 1: Do discounts have a statistically significant effect on the number of products customers order? If so, at what level(s) of discount?](#1) <br>
2. [Question 2: Does the Age of an Employee Have a Statistically Significant Effect on Their Sales Performance?](#2) <br>
3. [Question 3: Is there a statistically significant difference between the average order processing time in North America vs. Western Europe?](#3) <br>
4. [Question 4: Which Product Categories Generate Significantly More Revenue Than Average? Which Ones Significantly Less?](#4) <br>

## I. Import Libraries 

In [1]:
import sqlite3
import pandas as pd
import math
from scipy import stats
import seaborn as sns
import numpy as np

## II. Connect to the Database

In [2]:
connection = sqlite3.connect('Northwind_small.sqlite')
cursor = connection.cursor()

## II. Define Global Functions

These functions will be responsible for making tasks that need to be repeated throughout the notebook efficient and readable.

In [3]:
def sql_select_to_df(SQL_COMMAND, cursor=cursor):
    """returns output of SQL command in dataframe format"""
    results = cursor.execute(SQL_COMMAND).fetchall()
    df = pd.DataFrame(results)
    df.columns = [i[0] for i in cursor.description]
    return df

<a id="1"></a>
# 1. Do discounts have a statistically significant effect on the number of products customers order? If so, at what level(s) of discount? 

## 1.1 Do discounts have a statistically significant effect on the number of products customers order?

### 1.1.1 Source Data From Database

The two pieces of information we are looking to analyze in this first part of the question are both found in the OrderDetail table of the database, under the column names "Quantity" and "Discount" respectively.

In [4]:
orderDetail_df = sql_select_to_df("""select * from OrderDetail;""")
#orderDetail_df = orderDetail_df.iloc[:,-2:]
orderDetail_df

,Id,OrderId,ProductId,UnitPrice,Quantity,Discount
0,10248/11,10248,11,14.00,12,0.00
1,10248/42,10248,42,9.80,10,0.00
2,10248/72,10248,72,34.80,5,0.00
3,10249/14,10249,14,18.60,9,0.00
4,10249/51,10249,51,42.40,40,0.00
5,10250/41,10250,41,7.70,10,0.00
6,10250/51,10250,51,42.40,35,0.15
7,10250/65,10250,65,16.80,15,0.15
8,10251/22,10251,22,16.80,6,0.05
9,10251/57,10251,57,15.60,15,0.05


### 1.1.2 Extract Relevant Data to Test

Distinct sample groups must be extracted from the dataframe to pass to our chosen hypothesis test. For this first part of our question, where we are asking whether discounts have a significant effect on number of products ordered in general, the groups to be tested are: <br><br> 1. discounted orders <br>  2. non-discounted orders.

In [5]:
non_discounted_df = orderDetail_df.loc[orderDetail_df['Discount'] == 0]
discounted_df = orderDetail_df.loc[orderDetail_df['Discount'] != 0]

#### Discounted

In [121]:
discounted_orderQuantities = discounted_df['Quantity'].values
discounted_orderQuantities

array([ 35,  15,   6,  15,  40,  25,  15,  21,  50,  65,   6,  16,  15,
        21,  12,  60,  60,  36,  25,  12,  70,  15,  60,  20,  24,  15,
        60,  33,  12,   6,  15,  15,  20,   5,  45,  40,  36,  40,  15,
        10,   3,  20,  24,   2,  40,  30,  40,  30,  15,  25,  25,  30,
        40,  30,  25,  21,  70,  40,  80,  25,  50,  35,  30,  10,   8,
        20,  12,  50,  25,  40,  10,  16,  10,  40,   7,  25,   6,  48,
        18,  70,  20,  12,  40,   9,  24,  56,  40,  40,   4,  70,  36,
        50,   6,  15,  15,  18,  20,  77,  10,  20,  12,  50,  30,   8,
        10,  10,  20,  56,  70,  80,  54,  55,   5,  13,  35,  18,  15,
        20,   6,  12,  40,  70,  42,  80,  50,  42,  20,  20,   8,  16,
        20,  18,  20,   6,  10,  20,   8,  15,  20,  60,  40,  24,  25,
        42,   7,  70,  28,  70,  10,  18, 120,  21,  70,  30,  40,  30,
        30,  42,   5,   2,  25,  40,   9,  20,  18,   2,  36,  60,  20,
        20,   2,   8,  20,   4,  15,  10,  60,  49,  30,  10,  2

See above for an array representing the quantity of products ordered from each individual order detail that had a discount applied. There were 838 discounted purchase orders in total. 

This will be the dataset to be used in our hypothesis test as the experimental group.

#### Non-Discounted

In [127]:
non_discounted_orderQuantities = non_discounted_df['Quantity'].values
non_discounted_orderQuantities

array([12, 10,  5, ...,  1,  4,  2])

See above for an array representing the quantity of products ordered from each individual order detail that did not have a discount applied. There were 1317 discounted purchase orders in total. 

This will be the dataset to be used in our hypothesis test as the control group.

### 1.1.3 Test Hypothesis

#### Generate Test Question

We want to test whether there is simply **any** difference between the means in the quantity column of our **two** sample groups, so a **two-sample** t test will be chosen. The null and alternate hypotheses for this test are the following:


- $H_0$: There is no statistically significant difference between the number of products ordered in the discounted and non-discounted populations

- $H_\alpha$: There is a statistically significant difference between the number of products ordered in the discounted and non-discounted populations

#### Run Test

In [128]:
#Function
def run_twoSample_tTest(population1, population2, alpha):
    def get_t_statistic(pop1, pop2):
        """Calculates and returns t-statistic figure of two populations"""
        def variance(sample):
            sample_mean = np.mean(sample)
            return sum([(i - sample_mean)**2 for i in sample])
    
        def sample_variance(sample1, sample2):
            n_1, n_2 = len(sample1), len(sample2)
            var_1, var_2 = variance(sample1), variance(sample2)
            return (var_1 + var_2)/((n_1 + n_2)-2)
        
        pop1_mean, pop2_mean = np.mean(pop1), np.mean(pop2)
        samp_var = sample_variance(pop1, pop2)
        n_pop1, n_pop2 = len(pop1), len(pop2)
        t_numerator = pop1_mean - pop2_mean
        t_denominator = np.sqrt(samp_var * ((1/n_pop1)+(1/n_pop2)))
        return t_numerator / t_denominator
    
    t = get_t_statistic(population1, population2)
    p = stats.t.sf(abs(t), len(population1)+len(population2)-1) * 2
    
    if p < alpha:
        return 'reject null hypothesis'
    else:
        return 'fail to reject null hypothesis'

In [129]:
run_twoSample_tTest(discounted_orderQuantities, non_discounted_orderQuantities, 0.05)

'reject null hypothesis'

#### Conclusion
From the results of our t-test performed where we rejected the null hypothesis at an alpha level of 0.05, we can state with 95% confidence that there in fact **is a statistically significant difference** between number of discounted products ordered and non-discounted on the whole.

## 1.2 At what level of discount is there a statistically significant effect on the number of products customers order?

For this second part of our question we will need to repeat the above process, but instead of testing the mean of the **entire** discounted population against our non-discounted control population, we will need to use multiple experimental populations - i.e. separate experimental groups at  **each level of discount**.

### 1.2.1 Extract Relevant Data to Test

In order to create these separate experimmental groups, we first need to determine the unique discount rates that exist in the dataset.

In [10]:
discount_rates = discounted_df['Discount'].unique()
discount_rates = list(discount_rates)
discount_rates.sort()

print(discount_rates)

[0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.1, 0.15, 0.2, 0.25]


With this information, the separate groups can be created.

In [11]:
def get_discountLevel_data(rate):
    df = orderDetail_df.loc[orderDetail_df['Discount'] == rate]
    vals = df['Quantity'].values
    
    return vals

In [12]:
orderQuantities_01pct_discount = get_discountLevel_data(0.01)
orderQuantities_02pct_discount = get_discountLevel_data(0.02)
orderQuantities_03pct_discount = get_discountLevel_data(0.03)
orderQuantities_04pct_discount = get_discountLevel_data(0.04)
orderQuantities_05pct_discount = get_discountLevel_data(0.05)
orderQuantities_06pct_discount = get_discountLevel_data(0.06)
orderQuantities_10pct_discount = get_discountLevel_data(0.1)
orderQuantities_15pct_discount = get_discountLevel_data(0.15)
orderQuantities_20pct_discount = get_discountLevel_data(0.2)
orderQuantities_25pct_discount = get_discountLevel_data(0.25)

### 1.2.2 Run Hypothesis Tests

With our multiple experimental groups now created, we can test the following hypothesis test on each:
- $H_0$: There is **no** statistically significant difference between the number of products ordered at x% discounted and those with no discount

- $H_\alpha$: There **is** a statistically significant difference between the number of products ordered at x% discounted and those with no discount

In [13]:
def get_discountLevel_sampleSize(rate):
    val = get_discountLevel_data(rate)
    return len(val)

In [14]:
print('1 percent discount test outcome: ' + run_twoSample_tTest(orderQuantities_01pct_discount, non_discounted_orderQuantities, 0.05))
print('1 percent discount sample size: ' + str(get_discountLevel_sampleSize(0.01)))
print('------------------------------------')
print('2 percent discount test outcome: ' + run_twoSample_tTest(orderQuantities_02pct_discount, non_discounted_orderQuantities, 0.05))
print('2 percent discount sample size: ' + str(get_discountLevel_sampleSize(0.02)))
print('------------------------------------')
print('3 percent discount test outcome: ' + run_twoSample_tTest(orderQuantities_03pct_discount, non_discounted_orderQuantities, 0.05))
print('3 percent discount sample size: ' + str(get_discountLevel_sampleSize(0.03)))
print('------------------------------------')
print('4 percent discount test outcome: ' + run_twoSample_tTest(orderQuantities_04pct_discount, non_discounted_orderQuantities, 0.05))
print('4 percent discount sample size: ' + str(get_discountLevel_sampleSize(0.04)))
print('------------------------------------')
print('5 percent discount test outcome: ' + run_twoSample_tTest(orderQuantities_05pct_discount, non_discounted_orderQuantities, 0.05))
print('5 percent discount sample size: ' + str(get_discountLevel_sampleSize(0.05)))
print('------------------------------------')
print('6 percent discount test outcome: ' + run_twoSample_tTest(orderQuantities_06pct_discount, non_discounted_orderQuantities, 0.05))
print('6 percent discount sample size: ' + str(get_discountLevel_sampleSize(0.06)))
print('------------------------------------')
print('10 percent discount test outcome: ' + run_twoSample_tTest(orderQuantities_10pct_discount, non_discounted_orderQuantities, 0.05))
print('10 percent discount sample size: ' + str(get_discountLevel_sampleSize(0.10)))
print('------------------------------------')
print('15 percent discount test outcome: ' + run_twoSample_tTest(orderQuantities_15pct_discount, non_discounted_orderQuantities, 0.05))
print('15 percent discount sample size: ' + str(get_discountLevel_sampleSize(0.15)))
print('------------------------------------')
print('20 percent discount test outcome: ' + run_twoSample_tTest(orderQuantities_20pct_discount, non_discounted_orderQuantities, 0.05))
print('20 percent discount sample size: ' + str(get_discountLevel_sampleSize(0.20)))
print('------------------------------------')
print('25 percent discount test outcome: ' + run_twoSample_tTest(orderQuantities_25pct_discount, non_discounted_orderQuantities, 0.05))
print('25 percent discount sample size: ' + str(get_discountLevel_sampleSize(0.25)))
print('------------------------------------')

1 percent discount test outcome: fail to reject null hypothesis
1 percent discount sample size: 1
------------------------------------
2 percent discount test outcome: fail to reject null hypothesis
2 percent discount sample size: 2
------------------------------------
3 percent discount test outcome: reject null hypothesis
3 percent discount sample size: 3
------------------------------------
4 percent discount test outcome: fail to reject null hypothesis
4 percent discount sample size: 1
------------------------------------
5 percent discount test outcome: reject null hypothesis
5 percent discount sample size: 185
------------------------------------
6 percent discount test outcome: fail to reject null hypothesis
6 percent discount sample size: 1
------------------------------------
10 percent discount test outcome: reject null hypothesis
10 percent discount sample size: 173
------------------------------------
15 percent discount test outcome: reject null hypothesis
15 percent disco

### Conclusion

According to our tests, we can say with 95% confidence that the following discount rates make a statistically significant difference on the amount of products ordered:

- 0.05
- 0.10
- 0.15
- 0.20
- 0.25

Note: The null hypothesis was also rejected at discount rate 0.03 but had an insufficient sample size of only 3 to draw any significant conclusion from.

<a id="2"></a>
# 2. Does Employee Age Have a Statistically Significant Effect on Their Performance?

The two pieces of information we are looking to analyze in this first part of the question are found in the Order and Employee tables of the database respectively, under the column names "BirthDate" and "Total Orders".

## 2.1 Source Data from Database

In [130]:
orders_employees_df = sql_select_to_df("""SELECT O.EmployeeId, E.BirthDate, COUNT(O.Id) AS Total_Orders  
                                FROM [Order] AS O
                                JOIN Employee as E
                                ON O.EmployeeId = E.Id
                                GROUP BY O.EmployeeId
                        """)

orders_employees_df

,EmployeeId,BirthDate,Total_Orders
0,1,1980-12-08,123
1,2,1984-02-19,96
2,3,1995-08-30,127
3,4,1969-09-19,156
4,5,1987-03-04,42
5,6,1995-07-02,67
6,7,1992-05-29,72
7,8,1990-01-09,104
8,9,1998-01-27,43


## 2.2 Extract and Format Relevant Data for Testing

#### Ages

To begin investigating the age of each employee from only the birth dates given in the database, we need to estimate the current date of the dataset. We will use the date of the most recent order shipped to do this.

In [17]:
order_dates = sql_select_to_df("""SELECT ShippedDate FROM [Order]""")
print(order_dates)

    ShippedDate
0    2012-07-16
1    2012-07-10
2    2012-07-12
3    2012-07-15
4    2012-07-11
5    2012-07-16
6    2012-07-23
7    2012-07-15
8    2012-07-17
9    2012-07-22
10   2012-07-23
11   2012-07-25
12   2012-07-29
13   2012-07-30
14   2012-07-25
15   2012-07-31
16   2012-08-23
17   2012-08-12
18   2012-07-31
19   2012-08-06
20   2012-08-02
21   2012-08-09
22   2012-08-02
23   2012-08-30
24   2012-08-06
25   2012-08-12
26   2012-08-16
27   2012-08-09
28   2012-08-14
29   2012-08-13
..          ...
800  2014-04-30
801  2014-05-04
802  2014-05-05
803        None
804  2014-05-01
805  2014-04-29
806        None
807  2014-05-05
808  2014-05-01
809  2014-05-01
810        None
811        None
812  2014-05-04
813        None
814        None
815  2014-05-06
816  2014-05-04
817        None
818  2014-05-04
819  2014-05-06
820        None
821  2014-05-06
822        None
823        None
824        None
825        None
826        None
827        None
828        None
829        None

[830 ro

We can see above that the most recent order was shipped on May 6th, 2014, therefore we will assume the current year in our dataset is 2014.

In [18]:
dataset_yr = 2014

We now have enough information to obtain the age of each employee with the below process.

In [135]:
#Isolate list of birth dates
birth_dates = orders_employees_df['BirthDate'].values

#Extract year of birth
dates = []
for b in birth_dates:
    split_date = b.split('-')
    dates.append(split_date)

years = []
for d in dates:
    yr = int(d[0])
    years.append(yr)

#Subtract subtract each birth year from the current year to calculate age
ages = []
for y in years:
    age = dataset_yr - y
    ages.append(age)
    
age_sorted = sorted(ages)
print(f'Employee ages: {age_sorted}')
print(f'Corresponding Total Orders Made by Each Employee: {age_sorted}')

Employee ages: [16, 19, 19, 22, 24, 27, 30, 34, 45]


Great, we now have the employee ages component of our hypothesis. Now to acquire the total orders each has made.

#### Total Orders

In [140]:
employee_total_orders = list(orders_employees_df['Total_Orders'].values)
print(f'Employee total orders: {employee_total_orders}')

Employee total orders: [123, 96, 127, 156, 42, 67, 72, 104, 43]


## 2.3 Test Hypothesis

#### Formulate Hypothesis

Two options were considered on how to best test this hypothesis. The first option considered was to perform multiple cohen's d tests to determine whether there is a significant difference in the mean number of orders processed between each age. This option is more rigorous for selecting for each individual age, however is extremely limited by sample size (there is a maximum sample size of 2 for each age group "population"). The second option considered was to use an ANOVA test, which would better account for variance but require the data to be separated into groups which could potentially limit the granularity of the analysis.

Due to the extremity of the small sample size problem, an ANOVA was chosen with the following test hypotheses:
- $H_0$: There is **no** statistically significant difference between the number of total orders of each age group.

- $H_\alpha$: There **is** a statistically significant difference between the number of total orders of each age group.

#### Format data for testing

In order to run an ANOVA test on our data, it needs to be formatted into distinct groups. The groups chosen are under 20s, ages 20-29, and ages 30 and over.

In [142]:
orders_employees_df['Age'] = ages
orders_employees_df#

orders_age_df = orders_employees_df.iloc[:,2:]

under_twenties = orders_age_df.loc[orders_age_df['Age'] <20]
under_twenties_orders = under_twenties['Total_Orders'].values

twenties = orders_age_df.loc[(orders_age_df['Age'] >= 20) & (orders_age_df['Age'] < 30)]
twenties_orders = twenties['Total_Orders'].values

thirties_plus = orders_age_df.loc[(orders_age_df['Age'] >= 30)]
thirties_plus_orders = thirties_plus['Total_Orders'].values

print('population 1: (order figures of employees under age 20): ' + str(under_twenties_orders))
print('population 2: (order figures of employees between ages 20-29): ' + str(twenties_orders))
print('population 3: (order figures of employees of age 30 and over): ' + str(thirties_plus_orders))

population 1: (order figures of employees under age 20): [127  67  43]
population 2: (order figures of employees between ages 20-29): [ 42  72 104]
population 3: (order figures of employees of age 30 and over): [123  96 156]


#### Run Test

In [91]:
def run_ANOVA_test(pop1, pop2, pop3, alpha=0.05):
    
    #Calculate mean of all figures in all populations
    def get_grand_mean(population1, population2, population3):
        numerator = np.sum(population1) + np.sum(population2) + np.sum(population3)
        denominator = len(population1) + len(population2) + len(population3)
            
        grand_mean = numerator/denominator
        return grand_mean

    grand_mean = get_grand_mean(pop1, pop2, pop3)

    mean_1 = np.mean(under_twenties_orders)
    mean_2 = np.mean(twenties_orders)
    mean_3 = np.mean(thirties_plus_orders)
    
    #Calculate sum of squares for all figures in all populations
    def get_sum_squares_total(pop1, pop2, pop3):
        sst = []
    
        for i in pop1:
            sst_component_1 = (i - grand_mean)**2
            sst.append(sst_component_1)
        
        for j in pop2:
            sst_component_2 = (j - grand_mean)**2
            sst.append(sst_component_2)
        
        for k in pop3:
            sst_component_3 = (k - grand_mean)**2
            sst.append(sst_component_3)
    
        return sum(sst)

    sum_of_squares_total = get_sum_squares_total(pop1, pop2, pop3)
    
    #Calculate sum of squares within each population
    def get_sum_squares_within(pop1, pop2, pop3):
        ssw = []
    
        for i in pop1:
            ssw_component_1 = (i - mean_1)**2
            ssw.append(ssw_component_1)
        
        for j in pop2:
            ssw_component_2 = (j - mean_2)**2
            ssw.append(ssw_component_2)
        
        for k in pop3:
            ssw_component_3 = (k - mean_3)**2
            ssw.append(ssw_component_3)
    
        return sum(ssw)

    def get_ssw_degrees_of_freedom(m,n):    
        return m*(n-1)

    sum_of_squares_within = get_sum_squares_within(pop1, pop2, pop3)
    ssw_degrees_freedom = get_ssw_degrees_of_freedom(3, len(under_twenties_orders))
    
    #Calculate sum of squares between 
    def get_sum_squares_between(pop1, pop2, pop3):
        ssb = []
    
        for i in pop1:
            ssb_component_1 = (mean_1 - grand_mean)**2
            ssb.append(ssb_component_1)
        
        for j in pop2:
            ssb_component_2 = (mean_2 - grand_mean)**2
            ssb.append(ssb_component_2)
        
        for k in pop3:
            ssb_component_3 = (mean_3 - grand_mean)**2
            ssb.append(ssb_component_3)
    
        return sum(ssb)

    def get_ssb_degrees_of_freedom(m):    
        return m-1

    sum_of_squares_between = get_sum_squares_between(pop1, pop2, pop3)
    ssb_degrees_freedom = get_ssb_degrees_of_freedom(3)
    
    #Calculate F statistic
    def get_f_statistic(ss_between, df_between, ss_within, df_within):
        numerator = ss_between/df_between
        denominator = ss_within/df_within
        return numerator/denominator
    
    f_statistic = get_f_statistic(sum_of_squares_between, ssb_degrees_freedom, sum_of_squares_within, ssw_degrees_freedom)
    
    #Calculate critical F
    def get_critical_f(numerator, denominator):
        critical_f_data = [[19.00, 19.16,19.25,19.30,19.33,19.35,19.37,19.38], [9.55,9.28,9.12,9.01,8.94,8.89,8.85,8.81], [6.94,6.59,6.39,6.26,6.16,6.09,6.04,6.00],[5.79,5.41,5.19,5.05,4.95,4.88,4.82,4.77],[5.14,4.76,4.53,4.39,4.28,4.21,4.15,4.10],[4.74,4.35,4.12,3.97,3.87,3.79,3.73,3.68],[4.46,4.07,3.84,3.69,3.58,3.50,3.44,3.39],[4.26,3.86,3.63,3.48,3.37,3.29,3.23,3.18]] 
        critical_f_df = pd.DataFrame(critical_f_data,columns=[2,3,4,5,6,7,8,9])

        critical_f_df['index_new'] = [2,3,4,5,6,7,8,9]
        critical_f_df = critical_f_df.set_index('index_new')
        critical_f_val = critical_f_df.loc[denominator,numerator]
        
        return critical_f_val
    
    critical_f = get_critical_f(ssb_degrees_freedom, ssw_degrees_freedom)
    
    #---Return values---
    
    #F and critical F
    print(f'f-statistic: {f_statistic}')
    print(f'critical f value: {critical_f}')
    
    #Determine test outcome based on how F value compares to critical Fb
    if f_statistic > critical_f:
        print('test outcome: reject null hypothesis')
    else:
        print('test outcome: fail to reject null hypothesis')

run_ANOVA_test(under_twenties_orders, twenties_orders, thirties_plus_orders)

f-statistic: 1.9651173164421445
critical f value: 5.14
test outcome: fail to reject null hypothesis


#### Conclusion

It can be stated with 95% confidence that there is no statistically significant difference betweeen the amount of orders made by employees of different ages.

<a id="3"></a>
# 3. Is there a statistically significant difference between the average order processing time in North America vs. Western Europe? 

The two pieces of information we are looking to analyze in this question are found in the Customer and Order tables of the database respectively, under the column names "OrdeDate" and "shippedDate".

## 3.1 Source Data from Database

In [92]:
orders_cities = sql_select_to_df("""SELECT *
                                FROM [Customer] AS C
                                JOIN [Order] AS O
                                ON O.CustomerID = C.Id
                        """)

## 3.2 Extract Relevant Data

To find the processing time data to test for each region, we will need to find the difference between our 'OrderDate' and 'ShippedDate' columns.

####  Western Europe

In [143]:
#Extract relevant region
western_europe = orders_cities.loc[(orders_cities['ShipRegion'] == 'Western Europe') & (orders_cities['Region'] == 'Western Europe')]
western_europe_dates = western_europe.iloc[:,13:17]

#Convert orderDate, ShippedDate to datetime so they can be operated on
western_europe_dates.OrderDate = pd.to_datetime(western_europe_dates.OrderDate)
western_europe_dates.ShippedDate = pd.to_datetime(western_europe_dates.ShippedDate)

#Calculate processing time and return results
western_europe_dates['ProcessingTime'] = western_europe_dates.ShippedDate - western_europe_dates.OrderDate
processing_time_western_europe = western_europe_dates.ProcessingTime.dt.days
processing_time_western_europe = processing_time_western_europe.dropna()
processing_time_western_europe = processing_time_western_europe.values
processing_time_western_europe

array([12.,  5.,  7.,  2., 12.,  3.,  6., 10.,  8., 18.,  8.,  7., 10.,
        4.,  3.,  8.,  6.,  9.,  8.,  6.,  8., 29., 10., 10.,  7.,  5.,
        5.,  7.,  5., 10.,  5.,  6.,  7.,  8., 22.,  9., 12.,  9.,  7.,
       10., 11.,  3.,  8.,  3., 24., 21.,  3.,  3.,  8.,  8., 10.,  8.,
        6., 23.,  6.,  2.,  7., 10., 21., 35.,  4.,  6.,  8.,  7.,  5.,
        9., 20., 21.,  4.,  3.,  6.,  6.,  1.,  2.,  5.,  3., 21.,  8.,
        4.,  6., 12.,  8.,  3.,  8.,  7., 17., 27., 12.,  3.,  6., 24.,
       30.,  3.,  6., 21.,  2.,  2.,  4.,  2., 23.,  5., 25.,  6.,  4.,
        7.,  3.,  6.,  3.,  8.,  3.,  6., 17., 10.,  5., 17.,  4.,  7.,
        8., 35.,  4.,  7.,  9., 12.,  3.,  5.,  8.,  6.,  1.,  5.,  3.,
        6.,  7.,  8.,  1., 10., 17.,  3., 23., 10.,  7.,  8.,  2.,  7.,
        4.,  7.,  4.,  8., 19., 10.,  3.,  8.,  4.,  8.,  6.,  5.,  2.,
        2.,  9.,  8.,  1.,  7.,  6.,  5.,  9.,  2.,  7.,  9.,  2.,  6.,
        9.,  5.,  5.,  5.,  4., 10., 23.,  9.,  5.,  3., 29.,  7

Great, we now have the processing time data in Western Europe (in days) ready for testing.

#### North America

In [94]:
north_america = orders_cities.loc[(orders_cities['ShipRegion'] == 'North America') & (orders_cities['Region'] == 'North America')]
north_america_dates = north_america.iloc[:,13:17]

north_america_dates
north_america_dates.OrderDate = pd.to_datetime(north_america_dates.OrderDate)
north_america_dates.ShippedDate = pd.to_datetime(north_america_dates.ShippedDate)

north_america_dates['ProcessingTime'] = north_america_dates.ShippedDate - north_america_dates.OrderDate
processing_time_north_america = north_america_dates.ProcessingTime.dt.days

processing_time_north_america = processing_time_north_america.dropna()
processing_time_north_america = processing_time_north_america.values
processing_time_north_america

array([ 3.,  9., 29.,  4.,  6., 26.,  8.,  7.,  9., 11., 10.,  2.,  8.,
        4.,  4.,  7.,  4.,  3.,  7.,  7.,  3.,  4.,  6.,  4.,  9.,  9.,
       10.,  9.,  5., 11.,  9.,  4.,  8.,  7.,  3., 18., 32.,  6.,  4.,
        2., 20., 32., 10.,  8.,  7.,  7., 10.,  3.,  9., 35.,  2.,  6.,
        7., 25.,  2., 11.,  7.,  9., 10.,  7.,  7., 32.,  4.,  5., 21.,
        8.,  3.,  4.,  5.,  4.,  7.,  3.,  9., 12., 10.,  6., 11., 37.,
        9.,  6., 23.,  2.,  5.,  4.,  6.,  6.,  5., 19.,  8.,  2.,  5.,
        9.,  6., 26.,  6., 11., 12.,  4.,  8.,  5., 18.,  6., 14., 10.,
        8.,  6.,  9., 29.,  6.,  7.,  8., 19.,  4.,  8., 18.,  8.,  9.,
        8.,  1.,  7.,  2.,  3.,  9.,  9.,  9.,  1.,  4., 10.,  9.,  2.,
       12., 10.,  4., 10.,  2.,  8., 10.,  2.,  8.,  3.,  4., 10.,  7.,
        6.,  7.,  6.,  3.,  3.])

Great, we now have the processing time data in North America (in days) ready for testing.

### 3.3 Test Hypothesis

#### Run Test

Similarly to question 1, here we are testing for **any** statistically significant difference between **two** populations. Therefore we will again make use of the two-tail hypothesis test.

In [98]:
t_test = run_twoSample_tTest(processing_time_western_europe, processing_time_north_america,0.05)
t_test

'fail to reject null hypothesis'

#### Conclusion

It can be stated with 95% confidence that there is no statistically significant difference between the order processing time in North America and Western Europe

<a id="4"></a>
# 4.0 Which Product Categories Generate Significantly More Revenue Than Average? Which Ones Significantly Less?

The two pieces of information we are looking to analyze in this first part of the question are found in the Product and OrderDetail tables of the database respectively, under the column names "UnitPrice", "Quantity" and "CategoryID".

## 4.1 Source Data From Database

In [99]:
 product_orderDetails_df = sql_select_to_df("""SELECT *
                                FROM [Product] AS P
                                JOIN [OrderDetail] AS O
                                ON P.ID = O.ProductID
                        """)

product_orderDetails_df

,Id,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued,Id,OrderId,ProductId,UnitPrice,Quantity,Discount
0,11,Queso Cabrales,5,4,1 kg pkg.,21.00,22,30,30,0,10248/11,10248,11,14.00,12,0.00
1,42,Singaporean Hokkien Fried Mee,20,5,32 - 1 kg pkgs.,14.00,26,0,0,1,10248/42,10248,42,9.80,10,0.00
2,72,Mozzarella di Giovanni,14,4,24 - 200 g pkgs.,34.80,14,0,0,0,10248/72,10248,72,34.80,5,0.00
3,14,Tofu,6,7,40 - 100 g pkgs.,23.25,35,0,0,0,10249/14,10249,14,18.60,9,0.00
4,51,Manjimup Dried Apples,24,7,50 - 300 g pkgs.,53.00,20,0,10,0,10249/51,10249,51,42.40,40,0.00
5,41,Jack's New England Clam Chowder,19,8,12 - 12 oz cans,9.65,85,0,10,0,10250/41,10250,41,7.70,10,0.00
6,51,Manjimup Dried Apples,24,7,50 - 300 g pkgs.,53.00,20,0,10,0,10250/51,10250,51,42.40,35,0.15
7,65,Louisiana Fiery Hot Pepper Sauce,2,2,32 - 8 oz bottles,21.05,76,0,0,0,10250/65,10250,65,16.80,15,0.15
8,22,Gustaf's Knäckebröd,9,5,24 - 500 g pkgs.,21.00,104,0,25,0,10251/22,10251,22,16.80,6,0.05
9,57,Ravioli Angelo,26,5,24 - 250 g pkgs.,19.50,36,0,20,0,10251/57,10251,57,15.60,15,0.05


## 4.2 Obtain Data for Testing

#### 4.2.1 Extract Relevant Data

The first step will be to extract the two lists from our data that will later be used to formulate our experiment groups. They are: 
- categoryID 
- revenues

In [147]:
#Extract category IDs
category_IDs = product_orderDetails_df['CategoryId'].values
category_IDs = list(category_IDs)

#Extract revenues
prices = product_orderDetails_df.iloc[:,13].values
quantities = product_orderDetails_df['Quantity'].values

revenues = prices * quantities
revenues = list(revenues)
print({'categoryIDs':category_IDs ,'revenues':revenues})

{'categoryIDs': [4, 5, 4, 7, 7, 8, 7, 2, 5, 5, 2, 3, 4, 4, 4, 1, 3, 1, 6, 7, 1, 3, 8, 4, 6, 2, 3, 1, 2, 1, 2, 4, 3, 8, 8, 5, 3, 1, 3, 1, 2, 7, 5, 3, 1, 8, 7, 1, 8, 6, 1, 4, 8, 4, 1, 6, 4, 4, 4, 8, 1, 4, 3, 4, 4, 8, 4, 4, 8, 1, 4, 4, 1, 4, 8, 8, 7, 3, 2, 4, 2, 8, 6, 1, 6, 1, 3, 1, 1, 8, 5, 2, 3, 4, 4, 3, 2, 4, 1, 1, 8, 6, 1, 3, 3, 1, 8, 6, 3, 2, 5, 2, 6, 3, 2, 8, 2, 7, 3, 8, 1, 2, 1, 1, 6, 1, 4, 1, 5, 4, 3, 4, 1, 4, 1, 8, 4, 3, 3, 1, 2, 3, 8, 5, 6, 7, 1, 8, 2, 3, 3, 4, 4, 8, 6, 1, 8, 6, 6, 3, 3, 4, 1, 2, 2, 5, 1, 4, 3, 3, 5, 4, 7, 1, 6, 1, 8, 4, 8, 3, 1, 1, 8, 3, 1, 8, 1, 6, 7, 1, 4, 1, 5, 2, 3, 1, 3, 1, 8, 4, 2, 2, 8, 7, 4, 4, 2, 5, 1, 1, 4, 8, 8, 4, 2, 3, 3, 8, 1, 5, 3, 4, 6, 8, 5, 3, 7, 3, 4, 5, 3, 1, 4, 4, 7, 2, 5, 3, 8, 8, 4, 6, 8, 2, 6, 3, 8, 8, 1, 4, 4, 1, 4, 8, 6, 5, 3, 1, 2, 2, 2, 3, 5, 6, 5, 3, 1, 8, 1, 1, 5, 6, 3, 4, 1, 8, 2, 2, 1, 6, 4, 1, 1, 6, 1, 5, 4, 6, 4, 8, 3, 4, 1, 1, 8, 3, 4, 4, 7, 6, 1, 3, 6, 1, 4, 3, 7, 6, 4, 1, 1, 4, 4, 4, 2, 2, 1, 6, 2, 2, 3, 7, 5, 5, 6, 5, 1, 5,

Success. Now we need to boil this raw data into something we can use by formulating experimental groups for our tests.

#### 4.2.2 Get Revenue figures by Individual Categories

In order to answer our test question, we need to find a way to create individual experimental groups for each category. The first step towards doing this is to determine what unique categories exist in our dataset?

In [101]:
print(sorted(product_orderDetails_df['CategoryId'].unique()))

[1, 2, 3, 4, 5, 6, 7, 8]


OK. Next to generate the revenue figures for each.

In [103]:
def get_revenues_by_category(categoryNum, revenues_list, IDs_list):
    """Loops through revenues list and pushes relevant figure to list to be returned."""
    master_list = []
    iterator_count = 0
    
    for i in IDs_list:
        if i == categoryNum:
            master_list.append(revenues_list[iterator_count])
    
        iterator_count+=1
    
    return master_list
            

id1_revenues = get_revenues_by_category(1,revenues,category_IDs)
id2_revenues = get_revenues_by_category(2,revenues,category_IDs)
id3_revenues = get_revenues_by_category(3,revenues,category_IDs)
id4_revenues = get_revenues_by_category(4,revenues,category_IDs)
id5_revenues = get_revenues_by_category(5,revenues,category_IDs)
id6_revenues = get_revenues_by_category(6,revenues,category_IDs)
id7_revenues = get_revenues_by_category(7,revenues,category_IDs)
id8_revenues = get_revenues_by_category(8,revenues,category_IDs)

#### 4.2.3 Create Control and Experimental Groups for Testing

Next to generate our control groups for each experiment. This will be done with the below function.

In [151]:
def get_experimental_and_control_population(id_num, arr_of_groups):
    """Returns: 
        experimental dataset: category ID input (id_num argument),
        control dataset: rest of category ID groups i.e. selected id removed. (arr_of_groups argument)    
    """
    
    id_num_index = id_num - 1

    def get_control_population(generic):
        master_list = []
        for g in generic:
            master_list.append(g)
    
        return master_list

    control = get_control_population(arr_of_groups)

    return {'experimental':control.pop(id_num_index), 'control':control}

#Example: If you want control and experimental groups for revenues from categoryID 1
populations = get_experimental_and_control_population(1,[id1_revenues, id2_revenues, id3_revenues, id4_revenues, id5_revenues, id6_revenues, id7_revenues, id8_revenues])
populations

{'experimental': [604.8000000000001,
  54.0,
  304.0,
  86.4,
  760.0,
  252.0,
  288.0,
  100.8,
  532.0,
  240.0,
  216.0,
  919.9999999999999,
  475.2,
  43.2,
  43.2,
  186.0,
  21.6,
  57.6,
  56.0,
  648.0,
  1440.0,
  224.0,
  36.0,
  37.2,
  259.2,
  552.0,
  37.2,
  864.0,
  608.0,
  240.0,
  441.59999999999997,
  432.0,
  60.0,
  736.0,
  883.1999999999999,
  62.0,
  156.79999999999998,
  360.0,
  288.0,
  86.4,
  432.0,
  144.0,
  57.6,
  1008.0,
  310.0,
  380.0,
  4216.0,
  106.39999999999999,
  1472.0,
  364.79999999999995,
  216.0,
  720.0,
  37.2,
  216.0,
  4216.0,
  36.0,
  10540.0,
  172.8,
  90.0,
  36.0,
  112.0,
  2108.0,
  777.6,
  74.4,
  172.8,
  403.2,
  216.0,
  8432.0,
  288.0,
  360.0,
  54.0,
  112.0,
  54.0,
  360.0,
  576.0,
  380.0,
  432.0,
  504.0,
  504.0,
  144.0,
  345.6,
  201.6,
  10540.0,
  912.0,
  96.0,
  864.0,
  10329.2,
  288.0,
  259.2,
  152.0,
  148.8,
  224.0,
  684.0,
  115.2,
  86.4,
  72.0,
  300.0,
  288.0,
  736.0,
  24.8,
  110.39

With this function in place we can now create experimental and control groups for each product category

In [105]:
category1_data = get_experimental_and_control_population(1,[id1_revenues, id2_revenues, id3_revenues, id4_revenues, id5_revenues, id6_revenues, id7_revenues, id8_revenues])
category1_data_experimental = category1_data['experimental']
category1_data_control = category1_data['control']

category2_data = get_experimental_and_control_population(2,[id1_revenues, id2_revenues, id3_revenues, id4_revenues, id5_revenues, id6_revenues, id7_revenues, id8_revenues])
category2_data_experimental = category2_data['experimental']
category2_data_control = category2_data['control']

category3_data = get_experimental_and_control_population(3,[id1_revenues, id2_revenues, id3_revenues, id4_revenues, id5_revenues, id6_revenues, id7_revenues, id8_revenues])
category3_data_experimental = category3_data['experimental']
category3_data_control = category3_data['control']

category4_data = get_experimental_and_control_population(4,[id1_revenues, id2_revenues, id3_revenues, id4_revenues, id5_revenues, id6_revenues, id7_revenues, id8_revenues])
category4_data_experimental = category4_data['experimental']
category4_data_control = category4_data['control']

category5_data = get_experimental_and_control_population(5,[id1_revenues, id2_revenues, id3_revenues, id4_revenues, id5_revenues, id6_revenues, id7_revenues, id8_revenues])
category5_data_experimental = category5_data['experimental']
category5_data_control = category5_data['control']

category6_data = get_experimental_and_control_population(6,[id1_revenues, id2_revenues, id3_revenues, id4_revenues, id5_revenues, id6_revenues, id7_revenues, id8_revenues])
category6_data_experimental = category6_data['experimental']
category6_data_control = category6_data['control']

category7_data = get_experimental_and_control_population(7,[id1_revenues, id2_revenues, id3_revenues, id4_revenues, id5_revenues, id6_revenues, id7_revenues, id8_revenues])
category7_data_experimental = category7_data['experimental']
category7_data_control = category7_data['control']

category8_data = get_experimental_and_control_population(8,[id1_revenues, id2_revenues, id3_revenues, id4_revenues, id5_revenues, id6_revenues, id7_revenues, id8_revenues])
category8_data_experimental = category8_data['experimental']
category8_data_control = category8_data['control']

Groups generated. Ready for testing!

## 4.3 Test Hypothesis

#### Formulate Hypothesis

Because we are interested whether our sample mean is **significantly less than or greater than**  our population mean, a **one-sample** t test will be chosen. The null and alternate hypotheses for this test are the following:


- $H_0$: There is no statistically significant difference between the mean revenue generated by product category x and the mean of all other product categories.

- $H_\alpha$: product category x generated significantly more/less revenue than the mean of all other product categories

#### Run Tests

In [106]:
def one_sample_ttest(sample_data, population_data, alpha):
    
    #Population mean
    mu_calc_figures = []
        
    for p in population_data:
        for elem in p:
            mu_calc_figures.append(elem)
        
    mu = sum(mu_calc_figures)/len(mu_calc_figures)
    
    x_bar = sum(sample_data)/len(sample_data)
    
    #Sigma
    sample_data_arr = np.asarray(sample_data)
    sigma = np.std(sample_data_arr)
    
    #Degrees of Freedom
    df = len(sample_data) -1
    
    #Sample size
    n = len(sample_data)
    
    #Critical t value
    t_crit_positive = stats.t.ppf(1 - alpha, df=df)
    t_crit_negative = t_crit_positive * -1
    
    #T statistic
    t_stat_numerator = x_bar - mu
    t_stat_denominator = sigma/math.sqrt(n)
    t_stat = t_stat_numerator/t_stat_denominator
    
    if int(t_stat*100) in range(int(t_crit_negative*100), int(t_crit_positive*100)):
        return 'fail to reject the null hypothesis'
    elif int(t_stat*100) < int(t_crit_negative*100):
        return 'reject the null hypothesis. Sample mean is significantly less than population mean'
    elif int(t_stat*100) > int(t_crit_positive*100):
        return 'reject the null hypothesis. Sample mean is significantly greater than population mean'
    print(t_stat)

In [152]:
category1_test_outcome = one_sample_ttest(category1_data_experimental, category1_data_control, 0.05)
category2_test_outcome = one_sample_ttest(category2_data_experimental, category2_data_control, 0.05)
category3_test_outcome = one_sample_ttest(category3_data_experimental, category3_data_control, 0.05)
category4_test_outcome = one_sample_ttest(category4_data_experimental, category4_data_control, 0.05)
category5_test_outcome = one_sample_ttest(category5_data_experimental, category5_data_control, 0.05)
category6_test_outcome = one_sample_ttest(category6_data_experimental, category6_data_control, 0.05)
category7_test_outcome = one_sample_ttest(category7_data_experimental, category7_data_control, 0.05)
category8_test_outcome = one_sample_ttest(category8_data_experimental, category8_data_control, 0.05)

In [116]:
print(f'Test outcome of category 1: {category1_test_outcome}')
print(f'Test outcome of category 2: {category2_test_outcome}')
print(f'Test outcome of category 3: {category3_test_outcome}')
print(f'Test outcome of category 4: {category4_test_outcome}')
print(f'Test outcome of category 5: {category5_test_outcome}')
print(f'Test outcome of category 6: {category4_test_outcome}')
print(f'Test outcome of category 7: {category7_test_outcome}')
print(f'Test outcome of category 8: {category8_test_outcome}')

Test outcome of category 1: fail to reject the null hypothesis
Test outcome of category 2: reject the null hypothesis. Sample mean is significantly less than population mean
Test outcome of category 3: reject the null hypothesis. Sample mean is significantly less than population mean
Test outcome of category 4: reject the null hypothesis. Sample mean is significantly greater than population mean
Test outcome of category 5: reject the null hypothesis. Sample mean is significantly less than population mean
Test outcome of category 6: reject the null hypothesis. Sample mean is significantly greater than population mean
Test outcome of category 7: reject the null hypothesis. Sample mean is significantly greater than population mean
Test outcome of category 8: reject the null hypothesis. Sample mean is significantly less than population mean


#### Conclusion

We can see that the revenues generated by each category are quite polarizing. 7 of the 8 categories have a significant difference in mean than the rest of the population. 

The "winner" product categories generating above average revenue are:
- 4
- 6
- 7

The "par for the course" product category that generates about average revenue is:
- 1

The "not so great" categories that generate below average revenue are:
- 2
- 3
- 5
- 8